In [ ]:
import sys 
sys.path.append('..')

import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
import plotnine as gg

from ModelTools.regression.linear_model import LinearModel

In [ ]:
rng = np.random.default_rng(1)
n = 1000
data = pd.DataFrame({
    'x1':rng.normal(size=n),
    'x2':rng.normal(size=n),
    'x3':rng.normal(size=n),
    'x4':rng.normal(size=n)
}).assign(
    y=lambda dt:rng.normal(1+dt.x1+np.sin(dt.x2)+np.cos(dt.x3),scale=1),
)

data_train = data.iloc[:100,:]
data_test  = data.iloc[100:,:]


formula = 'y~x1+bs(x2,df=3)+bs(x3,df=3)+x4'
mod=LinearModel(formula,data=data,show_progress=True,fml_engine='patsy').fit(method='OLS',n_bootstrap=1000)

mod.summary()

# Check

In [ ]:
mod.check_resid_heterogeneity()

# Metric

In [ ]:
mod.metric()

In [ ]:
mod0 = LinearModel('y~x1+x2+x3+x4',show_progress=False,data=data_train).fit(n_bootstrap=0)
mod0.metric(new_data=data_test)

# Check

In [ ]:
mod.plot_check()

# Coef

In [ ]:
mod.coef()

In [ ]:
mod.plot_coef_dist()

# Prediction

In [ ]:
mod.prediction()

In [ ]:
mod.plot_prediction(data_grid={'x2':'line','x4':[1,10],'x3':[1,2],'x1':[1,2]},ci_type='obs',show_rug=False)

In [ ]:
mod.plot_prediction({'x2':'line100','x3':'line100'},plot_type='2d')

In [ ]:
mod.plot_all_prediction(ci_type='mean',free_y=True)

In [ ]:
mod.plot_all_prediction(ci_type='mean',free_y=True,color_by={'x4':'minmax'})

# Slope

In [ ]:
mod.slope()

In [ ]:
mod.plot_slope(data_grid={'x2':'line','x4':[1,10],'x3':[1,2],'x1':[1,2]},abline=None)

In [ ]:
mod.plot_all_slope(show_rug=True)

In [ ]:
mod.plot_all_slope(color_by={'x4':'minmax'})